# Minesweeper with propositional logic

## Goal of the project
The goal of the project is to implement an **intelligent agent** capable of autonomously playing the *Minesweeper* game using ***propositional logic***. The code base used is the one proposed by Harvard, available at [this link](https://cdn.cs50.net/ai/2020/spring/projects/1/minesweeper.zip).  
The structure of the application, including the user interface (UI), was already provided, however, the implementation of the agent was done entirely by me.

## Introduction to Minesweeper
Minesweeper is a single-player logic game whose goal is to discover all the cells in a grid that do not contain mines.

### Minesweeper Features:
- **Deterministic**: The player's actions do not affect the outcome of the game randomly. Every action has a predictable outcome.
- **Fully Observable**: Once a cell is discovered, the information about that cell is completely visible to the player.
- **Game Environment**: A grid* of hidden cells, some of which contain mines**, each discovered cell shows the number of adjacent mines.
    - '*' 6x6.
    - '**' in total there are 6 mines. 
  

## Propositional Logic
Propositional logic is a branch of logic that deals with *propositions* and their combinations through *logical connectives* such as **AND, OR, NOT**, etc. Through this logic it is possible to represent and manipulate knowledge.

### Components of Propositional Logic


**Propositions**

 Propositions are statements that can be **true** or **false**.
 - Examples of propositions: "Cell (1,1) is a mine".
 - Propositions are usually denoted by capital letters, for example:\( C_1_1 \).
 
 **Logical Connectives**
 
 Logical connectives are operators used to combine propositions and form more complex logical expressions.
 - **AND (∧)**: The conjunction of two propositions is true only if both propositions are true.
 - **OR (∨)**: The disjunction of two propositions is true if at least one of the propositions is true.
 - **NOT (¬)**: The negation of a proposition is true if the proposition is false.
 - **IMPLIES (→)**: The implication between two propositions is false only if the first proposition is true and the second is false.
 - **IFF (↔)**: The logical equivalence between two propositions is true if both propositions are both true or both false.

**Inference Rules**

In propositional logic, through rules, such as modus ponens or inference algorithms, such as **truth tables**, **forward chaining** and **backward chaining**, it is possible to deduce new propositions starting from existing propositions.



## `Cell` class
The `Cell` class represents a cell within the Minesweeper game, with information about its location, the presence of a mine, and the number of adjacent mines.

**Important note**: in addition to the various `set` and `get` methods, it was necessary to override the functions:

- `__eq__(self, other)`: To compare whether two cells are equal based on their position.
- `__hash__(self)`: to make the cell hashable, allowing it to be used as a key in a dictionary or as an element of a set.

In [28]:
class Cell:
    row = None
    col = None
    number = None
    has_mine = False
    def __init__(self,row=None,col=None):
        self.row = row
        self.col = col
    def set_has_mine(self,value):
        self.has_mine = value
    def set_number(self,n):
        self.number=n
    def get_has_mine(self):
        return self.has_mine
    def get_row(self):
        return self.row
    def get_col(self):
        return self.col
    def get_number(self):
        return self.number
    def __eq__(self, other):
        '''Override for self==other'''
        if isinstance(other, Cell):
            return self.row == other.row and self.col == other.col
        return False
    # Without the __hash__ function, comparing sets containing objects of the Cell class would not work correctly 
    # because the set comparison operation requires the elements of the set to be hashable. 
    # In Python, an object must implement the __hash__ method to be hashable, which means it can be used as a key in a dictionary or as an element in a set.
    def __hash__(self):
        return hash((self.row, self.col))

## `Minesweeper` class


The `Minesweeper` class manages the game i.e. the initialization of the field, the placement of mines and the calculation of the number of mines adjacent to each cell.

In [ ]:
import random
from cell import Cell
# It is a representation of the game
class Minesweeper():
    def __init__(self, height, width, mines):
        self.height = height
        self.width = width
        # This set represents the locations of the cells that have mines.
        # Is used, with the mines_found set, to check the victory
        self.mines = set()
        # No mines found
        self.mines_found = set()
        # Empty field
        self.field = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                row.append(Cell(i,j))
            self.field.append(row)
        # Add mines randomly
        count = 0
        while count != mines:
            i = random.randrange(self.height)
            j = random.randrange(self.width)
            #check each cell
            if not self.field[i][j].get_has_mine():
                self.mines.add(Cell(i, j))
                self.field[i][j].set_has_mine(True)
                count+=1

        
        # Added the number of nearby mines.
        self.compute_nearby_mines()
        self.print()

    def get_cell_from_field(self,cell):
        for i in range(self.height):
            for j in range(self.width):
                if self.field[i][j] == cell:
                    return self.field[i][j]
        return None

    def print(self):
        for i in range(self.height):
            print("--" * self.width + "-")
            for j in range(self.width):
                if self.field[i][j].get_has_mine():
                    print("|X", end="")
                else:
                    print(f"|{self.field[i][j].get_number()}", end="")
            print("|")
        print("--" * self.width + "-")
        string = "Mines are in: "
        for cell in self.mines:
            string+=f"({cell.get_row()},{cell.get_col()})"
        print(string)

    def is_mine(self,cell):
        return self.get_cell_from_field(cell).get_has_mine()

    def compute_nearby_mines(self):
        # Keep count of nearby mines
        directions = [
            (-1, -1), (-1, 0), (-1, 1),
            (0, -1),         (0, 1),
            (1, -1), (1, 0), (1, 1)
        ]
        for row in range(len(self.field)):
            for col in range(len(self.field[0])):
                if self.field[row][col].get_has_mine():
                    continue
                mine_count = 0
                for dr, dc in directions:
                    r, c = row + dr, col + dc
                    if 0 <= r < len(self.field) and 0 <= c < len(self.field[0]):
                        if self.field[r][c].get_has_mine():
                            mine_count += 1
                self.field[row][col].set_number(mine_count)

    def get_nearby_mines(self, cell):
        '''Returns the number of nearby mines of a given cell'''
        return self.get_cell_from_field(cell).number


## `Sentece` class

The use of traditional algorithms such as **model checking**, **forward chaining** and **backward chaining** has been avoided due to the high computational complexity associated with the numerous possible combinations in the Minesweeper game.


To show the complexity let's consider this example:
*If we suppose to know that **exactly one** of the eight variables is true*, this gives us a propositional logic sentence like the below:
    
    Or(
        And(A, Not(B), Not(C), Not(D), Not(E), Not(F), Not(G), Not(H)),
        And(Not(A), B, Not(C), Not(D), Not(E), Not(F), Not(G), Not(H)),
        And(Not(A), Not(B), C, Not(D), Not(E), Not(F), Not(G), Not(H)),
        And(Not(A), Not(B), Not(C), D, Not(E), Not(F), Not(G), Not(H)),
        And(Not(A), Not(B), Not(C), Not(D), E, Not(F), Not(G), Not(H)),
        And(Not(A), Not(B), Not(C), Not(D), Not(E), F, Not(G), Not(H)),
        And(Not(A), Not(B), Not(C), Not(D), Not(E), Not(F), G, Not(H)),
        And(Not(A), Not(B), Not(C), Not(D), Not(E), Not(F), Not(G), H)
    )
That is, a complicated expression and here we are assuming that we know that **only one** is a mine, but everything becomes much more complex if we suppose that there are 2,3,4 etc.

### Knowledge RepresentationTo avoid this complexity, I represent my sentences like this:

    {A, B, C, D, E, F, } = 1

Each sentence is made up of two parts: **a set of neighboring cells** on the grid and a **number** representing exactly how many of these cells are mines.

 - When the number of elements in the set is equal to the number assigned to that set then *we can infer that all the elements are
   mines*. 
 - When the number assigned to the set is 0, then *all elements in the set are not mines*.
  - When the number of elements is different from the number assigned to the set *it's no possible to generate new
   information*.


The `Sentence` class implements this behavior.



In [ ]:
from cell import Cell
class Sentence():
    '''
    This class represent logical sentences if the form {cell_A,cell_B,cell_C,cell_D} = 3, this means that in this four cells there are 3 mines. 
    A sentence consists of a set of cells, and a count of the number of those cells which are mines.
    '''
    def __init__(self, cells, count):
        self.cells = set(cells)
        self.count = count

    def __eq__(self, other):
        '''Override of == '''
        if isinstance(other, Sentence):
            return self.cells == other.cells and self.count == other.count
        return False
    
    def __str__(self):
        '''Override for str(sentence)'''
        empty = "{"
        empty += ",".join(f"({cell.row},{cell.col})" for cell in self.cells)
        empty += "}"
        empty += f" = {self.count}"
        return empty

    def known_mines(self):
        '''
        Returns the set of all cells in self.cells known to be mines.
        '''
        if len(self.cells) == self.count and self.count != 0:
            # means that all the cells are mines
            return self.cells
        else:
            # we cannot determine which specific cells are mines, so return an empty set
            return set()

    def known_safes(self):
        """
        Returns the set of all cells in self.cells known to be safe.
        """
        if self.count == 0:
            # if is 0 means that all the cells in the sentence are SAFE
            return self.cells
        else:
            # means that there are cells containing mines, but we don't know which ones so we return an empty set
            return set()

    def mark_mine(self, cell):
        """
        Updates internal knowledge representation given the fact that
        a cell is known to be a mine.
         
        """
        # If the cell is present in self.cells, it is removed from the set. Removing the cell from the set means that it will no longer be considered in the knowledge representation of this sentence.
        # this means that one of the cells thought to be a mine was confirmed as such, so the number of unknown mines in the sentence is reduced by one.
        if cell in self.cells:
            self.cells.remove(cell)
            self.count-=1
        # after this if for example we  call know_safes and returns a non empty set we know that all cells in the set are safe. 

    def mark_safe(self, cell):
        """
        Updates internal knowledge representation given the fact that
        a cell is known to be safe.
        """
        if cell in self.cells:
            self.cells.remove(cell)


## `Agent` class
The `Agent` class is responsible for implementing the intelligent agent that will infer whether a cell is safe or not and therefore play automatically.


We can consider functions:
 - `mark_safe(self, cell)` and `mark_mine(self, cell)` as **TELL** operations.
 - `make_safe_move(self)` as **ASK** operations.
 - `add_knowledge(self, cell, count)` as the **inference** operation.

## Inference process
In general the sentences in the knowledge base must refer to cells for which we do not yet know whether they are mine or safe cells, this means that, once we know whether a cell is a mine or not, we can update our sentences to simplify them and potentially draw new conclusions.
To insert a new sentence to add to the KB, we must consider two sets, `set1` and `set2` for which we have the number of mines: `number1` and `number2` respectively ,then we can construct the new sentence `set2 - set1 = number2 - number1`.
If we know, for example, that `{A, B, C} = 1` and `{A, B, C, D, E} = 2`, then logically we can infer that `{D, E} = 1`. Using these set operations we can do this.

The **idea** behind this process is to arrive at sentences where the size of the set is equal to the number associated with the set, this means that set is composed of cells that are **definitely mines**, or the number is 0, i.e. the **cells are safe** .
afe!

In [ ]:
from random import randrange
import random
import time
from cell import Cell
from sentence import Sentence
class Agent():
    """
    Thi class implement the agent that can play Minesweeper
    """
    # contains a set of all cells already clicked on
    moves_made = None
    # contains a set of all cells known to be safe
    safes = None
    # contains a set of all cells known to be mines
    mines = None
    # KB that is a list of all of the Sentences that the Agent knows to be true
    knowledge_base = None

    def __init__(self, height, width):
        # Set initial height and width
        self.height = height
        self.width = width
        self.moves_made = set()
        self.safes = set()
        self.mines = set()
        self.knowledge_base = []

    def mark_mine(self, cell):
        """
        Marks a cell as a mine, and updates all knowledge
        to mark that cell as a mine as well.
        """
        self.mines.add(cell)
        for sentence in self.knowledge_base:
            sentence.mark_mine(cell)

    def mark_safe(self, cell):
        """
        Marks a cell as safe, and updates all knowledge
        to mark that cell as safe as well.
        """
        self.safes.add(cell)
        for sentence in self.knowledge_base:
            sentence.mark_safe(cell)

    def make_safe_move(self):
        """
        Returns a safe cell to choose on the Minesweeper field.
        The move must be known to be safe, and not already a move
        that has been made.
        """
        safe_moves = self.safes - self.moves_made
        if safe_moves:
            choice = random.choice(list(safe_moves))
            print(f"The agent selected a safe movement = ({choice.row},{choice.col})")
            return choice
        else:
            return None

    def make_random_move(self):
        """
        Returns a move to make on the Minesweeper field.
        """
        # Space left on the field
        spaces_left = (self.height * self.width) - (len(self.moves_made) + len(self.mines))
        
        # If no spaces return None = no movement possible
        if spaces_left == 0:
            return None
        
        while True:
            i = randrange(self.height)
            j = randrange(self.width)
            cell = Cell(i,j)
            # have not already been chosen and are not known to be mines
            if (cell not in self.moves_made) and (cell not in self.mines):
                return cell
            
    def add_knowledge(self, cell, count):
        """
        Called when the Minesweeper field tells us, for a given
        safe cell, how many neighboring cells have mines in them.
        """
        # Mark the cell as a made and safe movement
        self.moves_made.add(cell)
        self.mark_safe(cell)
        #time.sleep(2)
        # Add new sentence to the Agent's KB based on the value of `cell` and `count`
        new_sentence = set()

        # Loop over adjacent cells
        for row in range(cell.row - 1, cell.row + 2):
            for col in range(cell.col - 1, cell.col + 2):
                adj_cell = Cell(row,col)
                # skip the cell itself and also when adj is know to be safe
                if (adj_cell == cell) or (adj_cell in self.safes):
                    continue
                
                # if is know that is mine the var count is decreased
                if adj_cell in self.mines:
                    #time.sleep(2)
                    count = count - 1
                    continue
                
                # within the limits of the playing field
                # adj_cell is not in safe set and not in mines set, so we have not info on it
                # we are not sure whether adj_cell is mine or safe so they are putted in a new sentece. 
                if 0 <= row < self.height and 0 <= col < self.width:
                    #print("Non ho certezze sulla cella ",row,col," quindi diventano una nuova frase da aggiungere:")
                    #time.sleep(2)
                    new_sentence.add(adj_cell)

        # just for printing purposes
        sentence = Sentence(new_sentence, count)
        print(f'The moving on cell: ({cell.row},{cell.col}) has added sentence = {sentence} to knowledge base' )
        
        # We add the information in the form {A,B,C,D} = count i.e. among those cells there are 'count' mines, but we don't yet know which ones they are
        self.knowledge_base.append(sentence)
        # Function to infer with the new sentence mines and safe cells, and  new knowledge
        new_inference = True
        while new_inference:
            new_inference = False
            mines = set()
            safes = set()
            # We collect all the cells that we know are safe and those that are mines
            for sentence in self.knowledge_base:
                # Returns the set of all cells in each setence known to be safes.
                safes = safes.union(sentence.known_safes())
                # Returns the set of all cells in each setence known to be mines.
                mines = mines.union(sentence.known_mines())

            # If there are safe cells or mines identified, update the Knowledge Base 
            # by marking them as such and set new_inference to True to continue the loop.
            if safes:
                new_inference = True
                for safe in safes:
                    self.mark_safe(safe)
            if mines:
                new_inference = True
                for mine in mines:
                    self.mark_mine(mine)

            # Remove any empty sentences from knowledge base:
            empty = Sentence(set(), 0)
            new_knowledge = []
            for sentence in self.knowledge_base:
                if sentence != empty:
                    new_knowledge.append(sentence)
            self.knowledge_base = new_knowledge

            # Try to infer new sentences from the current ones:
            for sentence_1 in self.knowledge_base:
                for sentence_2 in self.knowledge_base:

                    # Ignore when sentences are ==
                    if sentence_1.cells == sentence_2.cells:
                        continue

                    if sentence_1.cells == Cell() and sentence_1.count > 0:
                        print('Error - sentence with no cells and count created')
                        raise ValueError

                    # Create a new sentence if 1 is subset of 2, and not in KB:
                    # ex: sentence_1: {(1, 1), (1, 2)} = 1 
                    #     sentence_2: {(1, 1), (1, 2), (1, 3)} = 2 
                    #     sentece_2 - sentece_1 = {(1,3)} = 1
                    if sentence_1.cells.issubset(sentence_2.cells):
                        new_sentence_cells = sentence_2.cells - sentence_1.cells
                        new_sentence_count = sentence_2.count - sentence_1.count

                        new_sentence = Sentence(new_sentence_cells, new_sentence_count)

                        # Add to knowledge if not already in KB:
                        if new_sentence not in self.knowledge_base:
                            new_inference = True
                            print('New Inferred Knowledge: ', new_sentence, 'from', sentence_1, ' and ', sentence_2)
                            self.knowledge_base.append(new_sentence)

        # Print out AI current knowledge to terminal:
        print('Current AI KB length: ',len(self.knowledge_base))
        mines = Sentence(self.mines,len(self.mines))
        print('Known Mines: ', mines)
        remaining = Sentence(self.safes-self.moves_made,len(self.safes-self.moves_made))
        print('Safe Moves Remaining: ', remaining)
        print('====================================================')
        return self.mines


## Conclusions and future developments
The approach used, based on the use of propositional logic to infer the presence of bombs, has proven to be effective and efficient.
By avoiding traditional algorithms such as model checking, forward chaining and backward chaining, which are unusable for problems with high complexity due to combinatorial computation, the agent can perform inference in real time.

A possible improvement could be to use **probability calculation** (based on the number of free cells and information on mine and safe cells) for when the agent has no information on the cells and is therefore forced to make random movements.